Codigo seguindo o padrão do diagrama (atualizado, removendo o loggout) criado

In [ ]:
from ..database.database_module import DatabaseModule

class Authentication:

    def __init__(self, database_module: DatabaseModule):
        ...

    def authenticate_user(self, username: str, password: str) -> bool:
        ...

    def verify_password(self, input_password: str, hashed_password: str) -> bool:
        ...


Não passa no teste de autenticação com sucesso (não está implementado), autenticação implementada:

In [ ]:
from ..database.database_module import DatabaseModule
from ..user.user_management import UserManagement

class AuthenticationModule:

    def __init__(self, database_module: DatabaseModule):
        self.database_module = database_module
        self.user_management_module = UserManagement(self.database_module)

    def authenticate_user(self, username: str, password: str) -> bool:
        user = self.user_management_module.get_user(username)
        return self.verify_password(password, user.get_hashed_password())

    def verify_password(self, input_password: str, hashed_password: str) -> bool:
        ...


O código sem a verificação da password, assumindo que ela funciona, já passa no teste de login fracassado caso a senha esteja incorreta, mas falha quando é necessária a verificação da senha segundo o metodo implementado (funciona quando a senha está errada, pois é sempre None, mas nunca verifica uma senha como correta).
A implementação do hashing usando bcrypt é essa:

In [ ]:
from ..database.database_module import DatabaseModule
from ..user.user_management import UserManagement
import bcrypt

class AuthenticationModule:

    def __init__(self, database_module: DatabaseModule):
        self.database_module = database_module
        self.user_management_module = UserManagement(self.database_module)

    def authenticate_user(self, username: str, password: str) -> bool:
        user = self.user_management_module.get_user(username)
        return self.verify_password(password, user.get_hashed_password())

    def verify_password(self, input_password: str, hashed_password: str) -> bool:
        return bcrypt.checkpw(input_password.encode(), hashed_password.encode())
